# 따릉이 대여소 - 대여건수/반납건수/target 데이터 전처리

### 1. 따릉이_행정동 전처리

In [1]:
import numpy as np
import pandas as pd

In [2]:
#따릉이 대여소 데이터 불러오기
location = pd.read_csv('따릉이_행정동.csv',index_col=0)

#행정동_코드를 key로 사용하기위해 타입 변환
location = location.astype('str')

#읍면동코드 삭제
for i in range(len(location)):
    location['행정동_코드'][i]=location['행정동_코드'][i][0:8]
location

,대여소번호,행정동명,행정동_코드
0,102,서교동,11440660
1,103,망원1동,11440690
2,104,서교동,11440660
3,105,서교동,11440660
4,106,합정동,11440680
...,...,...,...
2581,5070,공항동,11500620
2582,5072,공항동,11500620
2583,5073,화곡3동,11500560
2584,5074,발산1동,11500611


### 2. 대여이력 전처리

In [3]:
#대여이력 불러오기
record=pd.read_csv('공공자전거 대여이력 정보_2107.csv', encoding='cp949')
record

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간,이용거리
0,SPB-32008,2021-07-01 22:06:23,3,중랑센터,0,2021-07-01 22:11:39,529,장한평역 8번 출구 앞,0,5,811.86
1,SPB-47263,2021-07-02 16:03:16,3,중랑센터,0,2021-07-02 16:08:00,529,장한평역 8번 출구 앞,0,4,0.00
2,SPB-54743,2021-07-04 22:01:50,3,중랑센터,0,2021-07-04 22:11:52,540,군자역 7번출구 베스트샵 앞,0,10,2450.00
3,SPB-44516,2021-07-06 16:29:58,3,중랑센터,0,2021-07-06 16:51:06,520,상왕십리역 4번 출구 앞,0,21,3608.08
4,SPB-52532,2021-07-07 16:01:48,3,중랑센터,0,2021-07-07 16:13:13,540,군자역 7번출구 베스트샵 앞,0,11,2410.32
...,...,...,...,...,...,...,...,...,...,...,...
3361555,SPB-33846,2021-07-31 22:23:24,9999,상담센터,0,2021-07-31 23:37:36,4173,이화약국 앞,0,74,11874.88
3361556,SPB-36747,2021-07-07 10:06:46,88888,영남배송,0,2021-07-07 10:14:43,236,문래동자이아파트 앞,0,7,837.62
3361557,SPB-38747,2021-07-07 10:06:32,88888,영남배송,0,2021-07-07 10:34:19,2080,빕스 대방점 앞,0,27,2535.67
3361558,SPB-52257,2021-07-21 09:51:12,88888,영남배송,0,2021-07-21 10:00:56,236,문래동자이아파트 앞,0,9,778.66


In [4]:
#컬럼명 수정
record.rename(columns={'대여 대여소번호':'대여대여소번호'},inplace=True)

In [5]:
#대여건수 count
out=record['대여대여소번호'].value_counts()
#데이터프레임화 시키고
out = pd.DataFrame(out)
#컬럼명 수정
out.rename(columns={'대여대여소번호':'대여건수'},inplace=True)
#인덱스로 들어간 대여소번호를 컬럼으로 꺼냄
out = out.rename_axis('대여소번호').reset_index()
#대여소번호는 id와 같으므로 타입변환
out['대여소번호']=out['대여소번호'].astype('str')

In [6]:
#따릉이_행정동 + 대여건수 join
cycle_merge=pd.merge(location,out,on='대여소번호',how='inner')
cycle_merge

,대여소번호,행정동명,행정동_코드,대여건수
0,102,서교동,11440660,4106
1,103,망원1동,11440690,3330
2,104,서교동,11440660,1657
3,105,서교동,11440660,1231
4,106,합정동,11440680,3200
...,...,...,...,...
2431,4864,송파2동,11710590,2443
2432,4865,방이2동,11710562,3749
2433,4867,가락본동,11710620,880
2434,4868,석촌동,11710600,1751


In [7]:
#결측치 확인
cycle_merge[cycle_merge['행정동명']=='nan']

,대여소번호,행정동명,행정동_코드,대여건수
284,436,nan,nan,1110
1196,1686,nan,nan,2985


In [8]:
#대여소번호를 통해 따로 확인해서 하나하나 값 넣어줌.
cycle_merge['행정동명'][284]='대흥동'
cycle_merge['행정동_코드'][284]='11440600'
cycle_merge['행정동명'][1196]='상계8동'
cycle_merge['행정동_코드'][1196]='11350700'

<ipython-input-8-47e9ee6441b8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cycle_merge['행정동명'][284]='대흥동'
<ipython-input-8-47e9ee6441b8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cycle_merge['행정동_코드'][284]='11440600'
<ipython-input-8-47e9ee6441b8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cycle_merge['행정동명'][1196]='상계8동'
<ipython-input-8-47e9ee6441b8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [9]:
#동별 대여건수 세기
cycle_out = cycle_merge.groupby(['행정동명','행정동_코드'],as_index=False).sum()
cycle_out

,행정동명,행정동_코드,대여건수
0,가락1동,11710631,9191
1,가락2동,11710632,7283
2,가락본동,11710620,5769
3,가리봉동,11530595,891
4,가산동,11545510,25549
...,...,...,...
411,효창동,11170580,1310
412,후암동,11170510,909
413,휘경1동,11230720,5099
414,휘경2동,11230730,11559


### 3. 반납 건수 전처리

In [10]:
#반납건수 count
in_=record['반납대여소번호'].value_counts()
#데이터프레임화 시키고
in_ = pd.DataFrame(in_)
#컬럼명 수정
in_.rename(columns={'반납대여소번호':'반납건수'},inplace=True)
#인덱스로 들어간 대여소번호를 컬럼으로 꺼냄
in_ = in_.rename_axis('대여소번호').reset_index()
#대여소번호는 id와 같으므로 타입변환
in_['대여소번호']=out['대여소번호'].astype('str')
in_

,대여소번호,반납건수
0,502,12563
1,2715,10674
2,207,10135
3,2102,9983
4,152,9884
...,...,...
2468,88888,4
2469,9991,1
2470,9985,1
2471,NaN,1


In [11]:
#따릉이_행정동 + 반납건수 join
cycle_merge2=pd.merge(location,in_,on='대여소번호',how='inner')
cycle_merge2=cycle_merge2.filter(['대여소번호','행정동명','행정동_코드','반납건수'])
cycle_merge2

,대여소번호,행정동명,행정동_코드,반납건수
0,102,서교동,11440660,4347
1,103,망원1동,11440690,3481
2,104,서교동,11440660,1694
3,105,서교동,11440660,1227
4,106,합정동,11440680,3339
...,...,...,...,...
2431,4864,송파2동,11710590,2519
2432,4865,방이2동,11710562,3937
2433,4867,가락본동,11710620,853
2434,4868,석촌동,11710600,1779


In [12]:
#결측치 확인
cycle_merge2[cycle_merge2['행정동명']=='nan']

,대여소번호,행정동명,행정동_코드,반납건수
284,436,nan,nan,1101
1196,1686,nan,nan,3102


In [13]:
#대여소번호를 통해 따로 확인해서 하나하나 값 넣어줌.
cycle_merge2['행정동명'][284]='대흥동'
cycle_merge2['행정동_코드'][284]='11440600'
cycle_merge2['행정동명'][1196]='상계8동'
cycle_merge2['행정동_코드'][1196]='11350700'

<ipython-input-13-051f94eb1911>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cycle_merge2['행정동명'][284]='대흥동'
<ipython-input-13-051f94eb1911>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cycle_merge2['행정동_코드'][284]='11440600'
<ipython-input-13-051f94eb1911>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cycle_merge2['행정동명'][1196]='상계8동'
<ipython-input-13-051f94eb1911>:5: SettingWithCopyWarning: 
A value is trying to be set on 

In [14]:
#동별 반납건수 세기
cycle_in = cycle_merge2.groupby(['행정동명','행정동_코드'],as_index=False).sum()
cycle_in

,행정동명,행정동_코드,반납건수
0,가락1동,11710631,9245
1,가락2동,11710632,7353
2,가락본동,11710620,5566
3,가리봉동,11530595,863
4,가산동,11545510,25578
...,...,...,...
411,효창동,11170580,1228
412,후암동,11170510,823
413,휘경1동,11230720,5111
414,휘경2동,11230730,11731


#### 5. 대여건수와 반납건수 데이터 합치기

In [15]:
#대여건수 + 반납건수 join
cycle_merge3=pd.merge(cycle_merge2,cycle_merge,on=['대여소번호','행정동명','행정동_코드'],how='inner')
cycle_merge3

,대여소번호,행정동명,행정동_코드,반납건수,대여건수
0,102,서교동,11440660,4347,4106
1,103,망원1동,11440690,3481,3330
2,104,서교동,11440660,1694,1657
3,105,서교동,11440660,1227,1231
4,106,합정동,11440680,3339,3200
...,...,...,...,...,...
2431,4864,송파2동,11710590,2519,2443
2432,4865,방이2동,11710562,3937,3749
2433,4867,가락본동,11710620,853,880
2434,4868,석촌동,11710600,1779,1751


In [16]:
#필요한 열만 추출
cycle_in_out= cycle_merge3.groupby(['행정동명','행정동_코드'],as_index=False).sum()
cycle_in_out

,행정동명,행정동_코드,반납건수,대여건수
0,가락1동,11710631,9245,9191
1,가락2동,11710632,7353,7283
2,가락본동,11710620,5566,5769
3,가리봉동,11530595,863,891
4,가산동,11545510,25578,25549
...,...,...,...,...
411,효창동,11170580,1228,1310
412,후암동,11170510,823,909
413,휘경1동,11230720,5111,5099
414,휘경2동,11230730,11731,11559


### 6. 대여소수 count

In [17]:
#대여소수 세기
대여소수=cycle_merge3['행정동_코드'].value_counts()
#데이터프레임화
대여소수 = pd.DataFrame(대여소수)
#컬럼명 수정
대여소수.rename(columns={'행정동_코드':'대여소수'},inplace=True)
#인덱스로 들어간 행정동_코드를 컬럼으로 꺼냄
대여소수 = 대여소수.rename_axis('행정동_코드').reset_index()

### 7. 반납건수, 대여건수를 동별 총 대여소수로 나누기 + 31일 나누기 -> 동별 대여소 하루 평균 반납건수, 대여건수

In [18]:
#대여소수+반납대여건수 데이터 join
cycle_in_out = pd.merge(left = cycle_in_out,right = 대여소수, on='행정동_코드', how='left')
cycle_in_out

,행정동명,행정동_코드,반납건수,대여건수,대여소수
0,가락1동,11710631,9245,9191,7
1,가락2동,11710632,7353,7283,5
2,가락본동,11710620,5566,5769,6
3,가리봉동,11530595,863,891,1
4,가산동,11545510,25578,25549,19
...,...,...,...,...,...
411,효창동,11170580,1228,1310,2
412,후암동,11170510,823,909,2
413,휘경1동,11230720,5111,5099,4
414,휘경2동,11230730,11731,11559,7


In [19]:
#반납건수/대여건수 대여소수로 나눠주기
for i in range(len(cycle_in_out)):
    cycle_in_out['반납건수'][i]=cycle_in_out['반납건수'][i]/cycle_in_out['대여소수'][i]
    cycle_in_out['대여건수'][i]=cycle_in_out['대여건수'][i]/cycle_in_out['대여소수'][i]

<ipython-input-19-3772434bd523>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cycle_in_out['반납건수'][i]=cycle_in_out['반납건수'][i]/cycle_in_out['대여소수'][i]
<ipython-input-19-3772434bd523>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cycle_in_out['대여건수'][i]=cycle_in_out['대여건수'][i]/cycle_in_out['대여소수'][i]


In [20]:
#중복값 확인함
cycle_in_out[cycle_in_out.duplicated('행정동명')]

,행정동명,행정동_코드,반납건수,대여건수,대여소수
241,신사동,11680510,1401,1398,3


In [21]:
#신사동 확인
cycle_in_out[cycle_in_out['행정동명']=='신사동']

,행정동명,행정동_코드,반납건수,대여건수,대여소수
240,신사동,11620685,2061,2002,4
241,신사동,11680510,1401,1398,3


In [22]:
#구가 다른데 행정동명이 같은것으로 확인 -> 따로 값 넣어줌.
cycle_in_out['행정동명'][240]='관악구_신사동'
cycle_in_out['행정동명'][241]='강남구_신사동'

<ipython-input-22-5e53ac05ab0f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cycle_in_out['행정동명'][240]='관악구_신사동'
<ipython-input-22-5e53ac05ab0f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cycle_in_out['행정동명'][241]='강남구_신사동'


In [23]:
#반납건수/대여건수를 31일로 나눠주기 -> 하루 평균 반납건수/대여건수 !!
cycle_in_out['반납건수']=cycle_in_out['반납건수'].astype('float')
cycle_in_out['대여건수']=cycle_in_out['대여건수'].astype('float')

cycle_in_out['반납건수']=round(cycle_in_out['반납건수']/31.0,2)
cycle_in_out['대여건수']=round(cycle_in_out['대여건수']/31.0,2)

cycle_in_out.rename(columns={'반납건수':'대여소당_하루평균_반납건수','대여건수':'대여소당_하루평균_대여건수'},inplace=True)
cycle_in_out

,행정동명,행정동_코드,대여소당_하루평균_반납건수,대여소당_하루평균_대여건수,대여소수
0,가락1동,11710631,42.58,42.35,7
1,가락2동,11710632,47.42,46.97,5
2,가락본동,11710620,29.90,31.00,6
3,가리봉동,11530595,27.84,28.74,1
4,가산동,11545510,43.42,43.35,19
...,...,...,...,...,...
411,효창동,11170580,19.81,21.13,2
412,후암동,11170510,13.26,14.65,2
413,휘경1동,11230720,41.19,41.10,4
414,휘경2동,11230730,54.03,53.26,7


In [24]:
#내보내기
cycle_in_out.to_csv("행정동별_반납건수_대여건수_대여소수_대여소당_하루평균.csv",mode="w",encoding='euc-kr',index=False)